# LoRA Fine-tuning with SmolLM2-135M using Unsloth

## Overview
This notebook demonstrates **LoRA (Low-Rank Adaptation)** fine-tuning of the SmolLM2-135M model using Unsloth.ai.

### What is LoRA?
- LoRA is a **parameter-efficient** fine-tuning method
- Updates only a small number of adapter parameters (< 1% of model)
- Much faster and memory efficient than full fine-tuning
- Achieves comparable performance with fewer resources

### Model Details
- **Model**: SmolLM2-135M (135 million parameters)
- **Method**: LoRA with r=16 (low rank)
- **Task**: Instruction following / Chat completion
- **Dataset**: Same 100 samples as Colab 1

### Key Difference from Colab 1:
- **Colab 1 (Full)**: r=256 → Updates ~36% of parameters (~78M params)
- **Colab 2 (LoRA)**: r=16 → Updates ~2% of parameters (~4M params)
- **Result**: LoRA is **faster** and uses **less memory**

## Step 1: Install Required Libraries

We'll install Unsloth and other dependencies needed for fine-tuning.

In [ ]:
# Install Unsloth for faster training
!pip install -q unsloth

# Install additional required packages
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

## Step 2: Import Libraries

Import all necessary libraries and disable wandb tracking.

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
import os

# Disable wandb tracking for simplicity
os.environ["WANDB_DISABLED"] = "true"

print("✓ All libraries imported successfully!")
print("✓ Weights & Biases tracking disabled")

## Step 3: Configure Model Parameters

Same configuration as Colab 1 for fair comparison.

In [ ]:
# Model configuration
max_seq_length = 512
dtype = None
load_in_4bit = True

# Same model as Colab 1
model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

print(f"Configuration:")
print(f"  Model: {model_name}")
print(f"  Max Sequence Length: {max_seq_length}")
print(f"  4-bit Quantization: {load_in_4bit}")

## Step 4: Load the Pre-trained Model

In [ ]:
# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("✓ Model loaded successfully!")
print(f"Model type: {type(model).__name__}")
print(f"Tokenizer vocab size: {len(tokenizer)}")

## Step 5: Prepare Model for LoRA Fine-tuning

**KEY DIFFERENCE**: Using r=16 (LOW rank) for LoRA!

### Comparison:
- **Colab 1**: r=256 (high rank) → ~78M trainable parameters
- **Colab 2**: r=16 (low rank) → ~4M trainable parameters

**Benefits of LoRA (r=16)**:
- ✅ Much faster training
- ✅ Less memory usage
- ✅ Smaller model files
- ✅ Easy to switch between adapters

In [ ]:
# Prepare model with LOW rank for LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LOW rank = Parameter-efficient LoRA!
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,  # Match with r
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("✓ Model prepared for LoRA fine-tuning!")
print(f"  Using LOW rank (r=16) for parameter-efficient training")
print(f"  This updates only ~2% of parameters (much less than Colab 1)")
print(f"\nTrainable parameters will be shown when training starts...")

## Step 6: Load and Prepare Training Dataset

Using the exact same dataset as Colab 1 for fair comparison.

In [ ]:
# Load same dataset as Colab 1
dataset = load_dataset("yahma/alpaca-cleaned", split="train")
dataset = dataset.select(range(100))  # Same 100 examples

print(f"✓ Dataset loaded: {len(dataset)} examples")
print("  (Same dataset as Colab 1 for fair comparison)")
print("\nSample example:")
print(dataset[0])

## Step 7: Define Chat Template and Formatting

Same Alpaca format as Colab 1.

In [ ]:
# Same prompt template as Colab 1
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

print("✓ Dataset formatted successfully!")
print("\nFormatted example (first 500 chars):")
print(dataset[0]["text"][:500] + "...")

## Step 8: Configure Training Arguments

Same training configuration as Colab 1 for fair comparison.

In [ ]:
# Same training configuration as Colab 1
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=60,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",  # Disable wandb
)

print("✓ Training arguments configured!")
print(f"  Total steps: {training_args.max_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Note: Same config as Colab 1, but with LoRA (r=16)")

## Step 9: Initialize the Trainer

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

print("✓ Trainer initialized successfully!")

## Step 10: Train the Model with LoRA

**Watch**: This should be FASTER than Colab 1 due to fewer trainable parameters!

In [ ]:
print("Starting LoRA training...")
print("This uses r=16 (LOW rank) - much fewer parameters than Colab 1!\n")

trainer_stats = trainer.train()

print("\n" + "="*60)
print("✓ LoRA Training completed!")
print("="*60)
print(f"Training time: {trainer_stats.metrics['train_runtime']:.2f} seconds")
print(f"Training loss: {trainer_stats.metrics['train_loss']:.4f}")
print(f"Samples per second: {trainer_stats.metrics['train_samples_per_second']:.2f}")
print(f"\nCompare this time with Colab 1 - LoRA should be faster!")

## Step 11: Test the LoRA Fine-tuned Model

In [ ]:
FastLanguageModel.for_inference(model)

test_instruction = "Explain what machine learning is in simple terms."
test_input = ""

test_prompt = alpaca_prompt.format(test_instruction, test_input, "")

print("Test Prompt:")
print(test_prompt)
print("\n" + "="*50 + "\n")

inputs = tokenizer([test_prompt], return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    use_cache=True,
    temperature=0.7,
    top_p=0.9,
)

response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print("LoRA Model Response:")
print(response)

## Step 12: More Test Examples

In [ ]:
def test_model(instruction, input_text=""):
    prompt = alpaca_prompt.format(instruction, input_text, "")
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        use_cache=True,
        temperature=0.7,
        top_p=0.9,
    )
    
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    print(f"\n{'='*80}")
    print(f"Instruction: {instruction}")
    if input_text:
        print(f"Input: {input_text}")
    print(f"\nResponse:")
    try:
        response_part = response.split('### Response:')[1].strip()
        print(response_part)
    except:
        print(response)
    print(f"{'='*80}\n")

print("Testing the LoRA fine-tuned model...\n")

test_model("Write a haiku about programming.")
test_model("What are the benefits of exercise?")
test_model("Summarize this text.", "Python is a high-level programming language known for its simplicity and readability.")

## Step 13: Save the LoRA Model

LoRA adapters are much smaller than full models!

In [ ]:
# Save LoRA adapters
model.save_pretrained("smollm2_135m_lora_adapters")
tokenizer.save_pretrained("smollm2_135m_lora_adapters")

print("✓ LoRA adapters saved to 'smollm2_135m_lora_adapters' directory")
print("\nNote: LoRA adapter files are MUCH smaller than full model!")
print("  - Full model: ~270MB")
print("  - LoRA adapters: ~10-20MB")
print("\nYou can load these adapters on top of the base model anytime!")

## Step 14: Comparison Summary

Let's compare LoRA (Colab 2) with Full Fine-tuning (Colab 1).

In [ ]:
print("\n" + "="*80)
print("COMPARISON: LoRA (Colab 2) vs Full Fine-tuning (Colab 1)")
print("="*80)
print("\n📊 Parameter Efficiency:")
print("  Colab 1 (Full, r=256):  ~78M trainable params (36.75%)")
print("  Colab 2 (LoRA, r=16):   ~4M trainable params (~2%)")
print("  Difference:             LoRA uses 95% FEWER trainable parameters!")

print("\n⏱️  Training Speed:")
print(f"  Colab 2 (LoRA):         {trainer_stats.metrics['train_runtime']:.2f} seconds")
print("  Expected: LoRA should be faster due to fewer parameters")

print("\n💾 Model Size:")
print("  Colab 1 (Full):         ~270MB (full model)")
print("  Colab 2 (LoRA):         ~10-20MB (adapters only)")
print("  Difference:             LoRA is 90% smaller!")

print("\n✅ Advantages of LoRA:")
print("  • Faster training")
print("  • Less memory usage")
print("  • Smaller model files")
print("  • Can have multiple adapters for different tasks")
print("  • Easy to share and deploy")

print("\n✅ Advantages of Full Fine-tuning:")
print("  • May achieve slightly better performance")
print("  • More comprehensive parameter updates")
print("  • Better for drastic task changes")
print("\n" + "="*80)

## Summary

### What We Did:
1. ✅ Loaded SmolLM2-135M model (same as Colab 1)
2. ✅ Configured for **LoRA fine-tuning** (r=16, low rank)
3. ✅ Used same dataset (100 examples) for fair comparison
4. ✅ Trained for 60 steps (same as Colab 1)
5. ✅ Tested the LoRA model
6. ✅ Saved LoRA adapters (much smaller!)

### Key Takeaways:
- **LoRA is parameter-efficient**: Updates only ~2% of parameters
- **LoRA is faster**: Fewer parameters = faster training
- **LoRA is smaller**: Adapter files are 90% smaller
- **LoRA is practical**: Easier to share and deploy
- **Performance**: Often comparable to full fine-tuning!

### When to Use LoRA vs Full Fine-tuning:
- **Use LoRA when**: Limited resources, need speed, multiple tasks
- **Use Full when**: Maximum performance needed, have resources

### Next Steps:
1. ✅ Record video comparing Colab 1 vs Colab 2
2. ✅ Highlight the efficiency gains of LoRA
3. ➡️ Move to **Colab 3** for DPO Reinforcement Learning

### Resources:
- LoRA Paper: https://arxiv.org/abs/2106.09685
- Unsloth Documentation: https://docs.unsloth.ai/
- LoRA Guide: https://docs.unsloth.ai/get-started/fine-tuning-llms-guide